# Support vector machines for Survival Analysis

In [1]:
import numpy as np
import math
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

## GBSG Data

In [2]:
from google.colab import files
uploaded = files.upload()

Saving GBSG2.csv to GBSG2.csv


In [165]:
GBSG = pd.read_csv('GBSG2.csv', sep = ";")

In [166]:
GBSG.head()

,horTh,age,menostat,tsize,tgrade,pnodes,progrec,estrec,time,cens
0,no,70,Post,21,II,3,48,66,1814,1
1,yes,56,Post,12,II,7,61,77,2018,1
2,yes,58,Post,35,II,9,52,271,712,1
3,yes,59,Post,17,II,4,60,29,1807,1
4,no,73,Post,35,II,1,26,65,772,1


In [167]:
GBSG.dtypes

horTh       object
age          int64
menostat    object
tsize        int64
tgrade      object
pnodes       int64
progrec      int64
estrec       int64
time         int64
cens         int64
dtype: object

In [168]:
d = GBSG.cens
y = GBSG.time

In [169]:
X = GBSG.drop(['time', 'cens'], axis = 1)
c_obj = X.select_dtypes(include='object').columns
X = pd.get_dummies(X, prefix=c_obj, drop_first = True)

In [170]:
X.head()

,age,tsize,pnodes,progrec,estrec,horTh_yes,menostat_Pre,tgrade_II,tgrade_III
0,70,21,3,48,66,0,0,1,0
1,56,12,7,61,77,1,0,1,0
2,58,35,9,52,271,1,0,1,0
3,59,17,4,60,29,1,0,1,0
4,73,35,1,26,65,0,0,1,0


In [249]:
from sklearn.model_selection import train_test_split
X = np.array(X)
d = np.array(d)
y = np.array(y)
X_train, X_test, y_train, y_test, d_train, d_test = train_test_split(X, y, d, test_size=0.5, random_state=1)

# RankSVM

**Comparable pairs**
A set $P = \{(i, j) | y_i > y_j \cap d_j = 1\}$ containing all comparable pairs. Note p is the number of elements in P.

**Ranking problem**
We need to find the rank function $f(Xi) = (X_iW)$ for the individual i with a constrain: Person has higher value of rank function will have longer survival time.

**Objective loss function with square Hinge-loss**
$$ f(W) = \frac{1}{2}W^TW+\frac{\gamma}{2} \sum_{i,j \in P} max(0, 1 - (X_iW - X_jW))^2$$


**Finding optimal coefficients vector W with sparse matrix A and diagonale matrix D**
$A \in R^{p n}$ with $A_{ki} = 1$ and $A_{kj} = -1$ if $(i,j) \in P$ and 0 ortherwise.

A diagonale matrix $D$ that indiates whether this pair is support vector: $ 1- (X_iW - X_jW) >0$

**The objective loss function** becomes:
$$ f(W) = \frac{1}{2}W^TW+\frac{C}{2} (\mathbb1 - AXW)D(\mathbb1 - AXW)^T$$

In [250]:
n = len(y_train)
# Construct set P
P = []
for i in range(n):
    for j in range(n):
        if ((d_train[j] == 1) & (y_train[i]>y_train[j])):
            P = np.append(P,[i, j], axis = 0)
P = np.reshape(P, (-1,2)).astype(int)

In [251]:
# Construct matrix A
p = P.shape[0]
A = np.zeros((p, n))
for i in range(p):
    A[i, P[i,0]] = 1
    A[i, P[i,1]] = -1

In [252]:
# Construct maxtrix D
# Initialize W
h = X_train.shape[1]
W = np.random.rand(h,1)
D = np.concatenate(A@X_train@W < 1)

#### Training by Newton's method

In [253]:
W = np.random.rand(h,1)
C = 0.00085; # best value on train set
for i in range(10):
    D = np.concatenate(A@X_train@W < 1)
    Aw = A[D, :]
    pw = np.sum(D==True)
    loss = 1/2 * W.T@W + C/2 *(pw + W.T@X_train.T@(Aw.T@Aw@X_train@W - 2*Aw.T@np.ones((pw,1))))
    grad = W + C * X_train.T @ (Aw.T@Aw@X_train@W - Aw.T@np.ones((pw,1)))
    hess = np.eye(h,h) + C * X_train.T @ Aw.T @ Aw @ X_train
    
    print(loss)
    W = W - np.linalg.inv(hess) @ grad
print('Number of true ranking prediction: ', np.sum(np.concatenate(A@X_train@W>0)), 'over ', p)
print('Harrell’s C-index:', np.sum(np.concatenate(A@X_train@W>0))/p)

[[302670.13747532]]
[[19.35760096]]
[[10.79753491]]
[[10.55102469]]
[[10.54919958]]
[[10.54919845]]
[[10.54919845]]
[[10.54919845]]
[[10.54919845]]
[[10.54919845]]
Number of true ranking prediction:  22278 over  31515
Harrell’s C-index: 0.706901475487863


**Prediction on test set**

In [254]:
n_t = len(y_test)
# Construct set P
P_t = []
for i in range(n_t):
    for j in range(n_t):
        if ((d_test[j] == 1) & (y_test[i]>y_test[j])):
            P_t = np.append(P_t,[i, j], axis = 0)
P_t = np.reshape(P_t, (-1,2)).astype(int)

In [255]:
# Construct matrix A
p_t = P_t.shape[0]
A_t = np.zeros((p_t, n_t))
for i in range(p_t):
    A_t[i, P_t[i,0]] = 1
    A_t[i, P_t[i,1]] = -1

In [256]:
print('Number of true ranking prediction on test set: ', np.sum(np.concatenate(A_t@X_test@W>0)), 'over ', p_t)
print('Harrell’s C-index:', np.sum(np.concatenate(A_t@X_test@W>0))/p_t)

Number of true ranking prediction on test set:  23101 over  34808
Harrell’s C-index: 0.6636692714318547


# Survival Regression SVM 

**Regression problem**
The function $f(Xi) = X_iW+b$ that modelize directly the exact time of an event.
For censored observation, we calculate the error of prediction only when the predicted survival time is before the time of censoring.

**Objective loss function with square Hinge-loss**
$$ f_{reg}(W, b) = \frac{1}{2}W^TW+\frac{\gamma}{2} \sum_{i=1}^n (\zeta_{W,b}(y_i, X_i, d_i) )^2$$

$$\zeta_{W,b}(y_i, X_i, d_i) = \begin{cases} max(0, y_i-X_iW-b) \quad\text{ if } d_i=0\\
y_i-X_iW-b \qquad \qquad \quad\text{if } d_i=1
\end{cases} $$
**Or in matrix form:**
$$ f_{reg}(\omega) = \frac{1}{2}\omega^T\omega+\frac{\gamma}{2} (y-X\omega)^TR_{\omega}(y-X\omega)$$
With $\omega=(b,W)$ and extending $X$ by a column of all ones, $R_{\omega}$ is a diagonale matrix with the i-th element being 1 if $(y_i > X_i\omega)$ or $(d_i=1)$, and zero otherwise.

#### Training by Newton's method

In [257]:
# Add vector 1 to X
# Initialize W
# Construct matrix R
Xn = np.c_[np.ones((n,1)), X_train]
hn = Xn.shape[1]
Wn = np.random.rand(hn,1)
Y = np.array(y_train).reshape((-1,1))
R = np.diag(np.logical_or((Y>Xn@Wn).flatten(),d_train==1))

In [258]:
Wn = np.random.rand(hn,1)
C = 14.5;
for i in range(6):
    R = np.diag(np.logical_or((Y > Xn @ Wn).flatten(),d_train==1))
    loss = 1/2 * Wn.T @ Wn + C/2 *(Y - Xn @ Wn).T @ R @ (Y - Xn @ Wn)
    grad = Wn - C * Xn.T @ R @ (Y - Xn @ Wn)
    hess = np.eye(hn,hn) + C * Xn.T @ R @ Xn
    
    print(loss)
    Wn = Wn - np.linalg.inv(hess) @ grad
print('Number of true ranking prediction: ', np.sum(np.concatenate(A@Xn@Wn>0)), 'over ', p)
print('Harrell’s C-index:', np.sum(np.concatenate(A@Xn@Wn>0))/p)

[[3.36436857e+09]]
[[6.78651661e+08]]
[[6.32196757e+08]]
[[6.32036181e+08]]
[[6.32035978e+08]]
[[6.32035978e+08]]
Number of true ranking prediction:  22045 over  31515
Harrell’s C-index: 0.6995081707123592


***Prediction on test set***

In [259]:
Xn_t = np.c_[np.ones((X_test.shape[0],1)), X_test]
print('Number of true ranking prediction on test set: ', np.sum(np.concatenate(A_t@Xn_t@Wn>0)), 'over ', p_t)
print('Harrell’s C-index:', np.sum(np.concatenate(A_t@Xn_t@Wn>0))/p_t)

Number of true ranking prediction on test set:  23537 over  34808
Harrell’s C-index: 0.6761951275568835


**Works to do next:**


*   Hybrid model that consider the ranking and regression at the same time. The loss objective function is defined as:
$$ f(W) = \frac{1}{2}W^TW+\frac{\gamma}{2} \big[(\alpha\sum_{i,j \in P} max(0, 1 - (X_iW - X_jW))^2+(1-\alpha)\sum_{i=1}^n (\zeta_{W,b}(y_i, X_i, d_i) )^2\big]$$

*   Kernel SVM for this survival problem which is much more powerful.

